##workspace.bronze.raw_device_message##
This table stores raw sensor readings from devices, including fields like device_id, timestamp, sensor_type, and distance. Each row represents a single sensor event, capturing the time, device, and measured distance (often as a string like "1cm"). This data is essential for feature engineering, as it provides the core signals (distance, time, device) needed for ML models.

In [0]:
%sql
-- List columns and data types for raw_device_message
DESCRIBE workspace.bronze.device_messages;

##workspace.bronze.rapid_step_test##
This table summarizes each rapid step test session, with fields such as start_time, stop_time, test_time, total_steps, and step_points (an array of step intervals). Each row represents a test window for a device and customer, defining the period during which steps were performed. This helps label sensor readings as "step" or "no_step" and provides context for aggregating features.

In [0]:
%sql
-- List columns and data types for raw_device_message
DESCRIBE workspace.bronze.rapid_step_tests;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.labeled_step_test (
  timestamp LONG COMMENT 'Exact time when the sensor reading was recorded (matches device_messages.timestamp).',
  device_id STRING COMMENT 'The device that produced the sensor reading. Connects readings to tests.',
  bronze_record_id LONG COMMENT 'Optional ID from the bronze device_messages row for traceability.',
  test_id STRING COMMENT 'Optional ID generated during ETL to group rows per rapid_step_tests.',
  sensor_type STRING COMMENT 'Type of sensor used (e.g., ultraSonicSensor). From device_messages.',
  distance_cm DOUBLE COMMENT 'Distance measurement in cm (converted from string). Core ML feature.',
  step_label STRING COMMENT 'Label indicating if timestamp is within a rapid_step_tests window. Values: "step"/"no_step".',
  source STRING COMMENT 'Origin of the row (e.g., "device"). Useful for troubleshooting.',
  start_time LONG COMMENT 'Start time of Rapid Step Test window.',
  stop_time LONG COMMENT 'End time of Rapid Step Test window.',
  customer STRING COMMENT 'Customer associated with the rapid step test session.',
  test_time LONG COMMENT 'Duration of the rapid step test session.',
  total_steps INT COMMENT 'Total steps recorded in the rapid step test.'
)
USING DELTA;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.labeled_step_test AS
SELECT
  dm.*,
  rt.customer,
  rt.start_time,
  rt.stop_time,
  rt.test_time,
  rt.total_steps
FROM workspace.bronze.device_messages dm
LEFT JOIN workspace.bronze.rapid_step_tests rt
  ON dm.device_id = rt.device_id
  AND dm.timestamp BETWEEN rt.start_time AND rt.stop_time;

In [0]:
%sql
SELECT 
  column_name AS `Column Name`, 
  data_type AS `Data Type`,
  comment AS Comment
FROM 
  workspace.information_schema.columns
WHERE 
  table_schema = 'silver'
  AND table_name = 'labeled_step_test';
